In [6]:
from sqlalchemy import create_engine
import pandas as pd
import pymysql
import datetime

db_connection = pymysql.connect(host='localhost', port=3306, user='root', password='root', db='data_sensors')

In [19]:
df = pd.read_sql('SELECT * FROM air_quality_sensors', con=db_connection)
df.head(5)

,id,entry_id,eCO2,eTVOC,Temperature,Air_pressure,Humidity,temperature_,Controller_temperature,G,date_creation,time_float
0,1,34773,698.5,1654.5,20.85,971.00,39.94,21.06,41.90,4.36,2022-04-06,21.0633
1,2,34774,608.2,1663.7,20.03,970.96,42.40,19.99,41.07,4.32,2022-04-06,21.2553
2,3,34776,417.2,682.9,17.71,970.91,47.25,17.90,38.38,2.92,2022-04-06,21.4483
3,4,34777,500.8,591.2,19.33,970.78,42.97,19.30,40.14,2.93,2022-04-06,21.6403
4,5,34778,527.4,670.9,19.67,970.77,42.13,19.87,40.78,3.05,2022-04-06,21.8325


In [21]:
#verificando hora
r = datetime.timedelta(hours=5.74861 - 3)
str(r)

'2:44:54.996000'

c:\Users\igorf\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


TypeError: float() argument must be a string or a number, not 'datetime.timedelta'

In [3]:
# Quantos dados? 
df.shape

(8066, 12)

Os tipos Object são iguais String

In [5]:
# Quais os tipos? Dados faltantes? 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8066 entries, 0 to 8065
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      8066 non-null   int64  
 1   entry_id                8066 non-null   int64  
 2   eCO2                    8066 non-null   float64
 3   eTVOC                   8066 non-null   float64
 4   Temperature             8066 non-null   float64
 5   Air_pressure            8066 non-null   float64
 6   Humidity                8066 non-null   float64
 7   temperature_            8066 non-null   float64
 8   Controller_temperature  8066 non-null   float64
 9   G                       8066 non-null   float64
 10  date_creation           8066 non-null   object 
 11  time_float              8066 non-null   float64
dtypes: float64(9), int64(2), object(1)
memory usage: 756.3+ KB


In [6]:
#de quantos dias diferentes
df['date_creation'].nunique()

65

In [7]:
#analise dos dados
df.describe()

,id,entry_id,eCO2,eTVOC,Temperature,Air_pressure,Humidity,temperature_,Controller_temperature,G,time_float
count,8066.000000,8066.000000,8066.000000,8066.000000,8066.000000,8066.000000,8066.000000,8066.000000,8066.000000,8066.000000,8066.000000
mean,4033.500000,38740.337590,537.121535,2715.315572,20.679116,983.857613,46.503495,20.646332,41.298509,3.173923,12.018937
std,2328.597969,2326.778672,110.125333,3400.641419,2.159467,6.860016,11.153372,2.132563,2.167896,2.410242,6.943144
min,1.000000,34773.000000,406.500000,0.000000,11.650000,960.320000,21.220000,10.690000,31.340000,0.610000,0.002222
25%,2017.250000,36722.250000,459.500000,209.325000,19.522500,980.750000,38.770000,19.580000,40.190000,1.380000,5.997010
50%,4033.500000,38739.500000,505.400000,1416.900000,20.655000,984.955000,45.470000,20.690000,41.320000,2.660000,12.040000
75%,6049.750000,40755.750000,587.100000,4070.525000,22.077500,988.890000,53.310000,21.960000,42.690000,4.310000,18.038550
max,8066.000000,42772.000000,1141.000000,26224.500000,26.820000,997.240000,84.250000,26.660000,47.380000,18.510000,23.999200


##### Nao vai limpeza dos dados porque estao todos preenchidos pois o banco so aceita NOT NULL

In [12]:
df['date_c'] = pd.to_datetime(df['date_creation'])
df['date_c'].iloc[0]

Timestamp('2022-04-06 00:00:00')

In [6]:
#tratando data para treinamento
df['date_c'] = pd.to_datetime(df['date_creation'])
df['date_c'] = df['date_c'].map(datetime.datetime.toordinal)

In [7]:
del df['date_creation']
del df['id']
del df['entry_id']

In [10]:
df.head(1)

,eCO2,eTVOC,Temperature,Air_pressure,Humidity,temperature_,Controller_temperature,G,time_float,date_c
0,698.5,1654.5,20.85,971.0,39.94,21.06,41.9,4.36,21.0633,738251


### Modelagem

In [11]:
import numpy as np
np.array(df.filter(['eTVOC', 'eCO2']))

array([[1654.5,  698.5],
       [1663.7,  608.2],
       [ 682.9,  417.2],
       ...,
       [ 915.1,  610. ],
       [ 927.5,  612.1],
       [ 962.1,  616.8]])

In [8]:
# Separação de DADOS e LABEL
X = df.drop(columns=['eTVOC', 'eCO2'])
Y = df.filter(['eTVOC', 'eCO2'])

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [10]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size = 0.2,
                                                    random_state = 42)

In [11]:
from sklearn.linear_model import LinearRegression
# Treinando o modelo
modelo = LinearRegression(n_jobs=20,positive=True, normalize=True)
modelo.fit(X_train, Y_train)

c:\Users\igorf\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


LinearRegression(n_jobs=20, normalize=True, positive=True)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, classification_report

In [13]:
# Fazendo as predições
Y_pred = modelo.predict(X_test)

In [34]:
Y_pred.iloc[742]

eTVOC_pred    1187.088078
eCO2_pred      528.917830
Name: 742, dtype: float64

In [14]:
Y_pred = pd.DataFrame(Y_pred, columns=['eTVOC_pred', 'eCO2_pred'])
frames = [Y_test,Y_pred]
v = pd.concat(frames, axis=1, join="inner")

v

,eTVOC,eCO2,eTVOC_pred,eCO2_pred
742,1025.0,645.2,1187.088078,528.917830
1302,1445.1,424.2,5265.983476,582.637399
1168,2750.9,536.9,4156.583900,540.752517
429,64.8,459.9,2402.484420,520.814518
233,579.7,952.3,3430.097411,552.170821
...,...,...,...,...
1608,9.7,524.5,1323.093200,515.795591
418,1917.5,719.6,5525.881296,602.744180
293,896.6,637.1,248.232031,503.713456
1149,1708.2,489.0,2089.199458,508.910759


### Avaliação

In [15]:
# Erro quadrático médio
print('eTVOC: ', mean_squared_error(Y_test['eTVOC'], Y_pred['eTVOC_pred']))
print('eCO2: ', mean_squared_error(Y_test['eCO2'], Y_pred['eCO2_pred']))

eTVOC:  3941413.104071415
eCO2:  10593.609063585385


In [16]:
# Erro absoluto médio
print('eTVOC: ', mean_absolute_error(Y_test['eTVOC'], Y_pred['eTVOC_pred']))
print('eCO2: ', mean_absolute_error(Y_test['eCO2'], Y_pred['eCO2_pred']))

eTVOC:  1420.7318482937687
eCO2:  76.65526232137644


In [17]:
# R²
print('eTVOC: ', r2_score(Y_test['eTVOC'], Y_pred['eTVOC_pred']))
print('eCO2: ', r2_score(Y_test['eCO2'], Y_pred['eCO2_pred']))

eTVOC:  0.6591299943827983
eCO2:  0.18026769735499237


In [4]:
from sklearn import model_selection
import pickle

In [2]:
# salvando model
filename = 'finalized_model.sav'
#pickle.dump(modelo, open(filename, 'wb'))

In [5]:
with open(filename , 'rb') as f:
    loaded_model = pickle.load(f)

In [26]:
Y_pred = loaded_model.predict(X_test)
Y_pred = pd.DataFrame(Y_pred, columns=['eTVOC_pred', 'eCO2_pred'])

In [27]:
# R²
print('eTVOC: ', r2_score(Y_test['eTVOC'], Y_pred['eTVOC_pred']))
print('eCO2: ', r2_score(Y_test['eCO2'], Y_pred['eCO2_pred']))

eTVOC:  0.6591299943827983
eCO2:  0.18026769735499237
